In [3]:
!pip3 install transformers databench-eval

In [4]:
import os
import zipfile
import ast
from tqdm.notebook import tqdm

from datasets import Dataset 
from databench_eval import Runner, Evaluator, utils

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"
access_token = ""

tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    token=access_token
)


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [6]:
import pandas as pd

qa_df = pd.read_csv('/kaggle/input/semvaltask8/competition/test_qa.csv')
base_data_path = '/kaggle/input/semvaltask8/competition/'

In [7]:
import textwrap
import numpy as np 

def postprocess_response(response: str, provided_dataset: str) -> str:
    code_from_llm = response.strip()
    if code_from_llm.startswith("```python"):
        code_from_llm = code_from_llm[len("```python"):].strip()
    if code_from_llm.startswith("```"):
         code_from_llm = code_from_llm[len("```"):].strip()
    if code_from_llm.endswith("```"):
        code_from_llm = code_from_llm[:-len("```")].strip()


    indented_code = textwrap.indent(code_from_llm, '    ')

    provided_df = pd.read_parquet(base_data_path + provided_dataset + '/sample.parquet') 
    func_code_string = code_from_llm
    
    try:

        exec_scope = {'pd': pd, 'np': np, 'df': provided_df }
        
    except Exception as e:
        print('data load error')
        return f"__DATA_LOAD_ERROR__: {e}"

    try:
        exec(func_code_string, exec_scope) # run the generated code with necessary scopes

        answer_func = exec_scope.get('generated_answer_func')
        if callable(answer_func):
            try:
                final_answer = answer_func(exec_scope['df'])
                
                if isinstance(final_answer, (pd.Series, pd.DataFrame)):
                    if isinstance(final_answer, pd.Series):
                        formatted_answer = str(final_answer.tolist())
                    else:
                         formatted_answer = "__CODE_ERROR__: Returned DataFrame, expected scalar/list/bool"
                elif isinstance(final_answer, list):
                    formatted_answer = str(final_answer)
                elif isinstance(final_answer, (int, float, bool, str, np.generic)):
                     formatted_answer = str(final_answer)
                elif final_answer is None:
                     formatted_answer = "__CODE_ERROR__: Returned None"
                else:
                    try:
                        formatted_answer = str(final_answer)
                    except Exception as fmt_e:
                         formatted_answer = f"__FORMAT_ERROR__: Could not convert {type(final_answer)} to string: {fmt_e}"

                if isinstance(formatted_answer, str) and "\n" in formatted_answer:
                    formatted_answer = formatted_answer.split("\n")[0]
                print('ans: ', formatted_answer)
                return formatted_answer

            except Exception as exec_e:
                print('exec error', exec_e)
                return f"__CODE_EXEC_ERROR__: {exec_e}"
        else:
            print('not callable')
            return "__CODE_DEF_ERROR__: generated_answer_func not callable"

    except SyntaxError as syn_e:
         return f"__CODE_SYNTAX_ERROR__: {syn_e}"
    except Exception as e:
        return f"__EXEC_ERROR__: {e}"

In [8]:
def generate_prompt_messages(row: dict) -> list:
    question = row["question"]
    dataset_name = row["dataset"]

    try:
        df_sample = pd.read_parquet(base_data_path + dataset_name + '/sample.parquet') 
        columns = df_sample.columns.tolist()
        dtypes = df_sample.dtypes.to_dict()
        head_csv = df_sample.head(5).to_csv(index=False)
        del df_sample 
    except Exception as e:
        print(f"Warning: Could not load sample for {dataset_name} for prompt generation: {e}")
        columns = ["unknown"]
        dtypes = {}
        head_csv = "Error loading sample data"

    messages = [
        {"role": "system", "content": "You are a python programmer who takes a pandas dataset, a question and returns the code to answer that question based on solely the dataset."},
            {"role": "user", "content": f"""Question: {question}
# Table schema
Dataset columns: {columns}
Data Types: {dtypes}
# Sample Rows
First 5 rows: {head_csv}
# Output format:
Boolean - if asking a yes-no, true-false question
Cell Value - if the answer should be a value from dataframe
Number - if the answer should be a numerical computed value or just a numerical value from dataframe
List[cell values] - if asked for a list of values from cells of the dataframe
List[number] - if asked for a list of numerical values either computed or directly from cells
# Rules:
You have access to pandas and numpy only
Be careful about types and do type conversion when necessary
Don't return any explanation, formatting or comment
You should write short, concise and efficient code

Now, write the function below with proper indentation to return the answer to the question.

def generated_answer_func(df: pd.DataFrame):
""" },
    ]
    return messages

In [9]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
def llama3_model_call(prompt_messages_batch: list) -> list:

    results = []
    for messages in prompt_messages_batch:
        try:
            text = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                tokenize=False,
            )
            model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

            # attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=model.device)
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=512,
                do_sample=False
            )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            
            llm_code = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            
            # outputs = model.generate(
            #     input_ids,
            #     attention_mask=attention_mask, 
            #     max_new_tokens=350,
            #     eos_token_id=terminators,
            #     pad_token_id=tokenizer.pad_token_id, 
            #     do_sample=False, 
            #     # temperature=0.6, # Use if do_sample=True
            #     # top_p=0.9,       # Use if do_sample=True
            # )
 
            # response_ids = outputs[0][input_ids.shape[-1]:]
            # llm_code = tokenizer.decode(response_ids, skip_special_tokens=True)
            results.append(llm_code)

            del model_inputs, generated_ids
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"Error during model generation: {e}")
            results.append(f"__CODE_GEN_ERROR__: {e}")

    return results

In [10]:
utils.COMPETITION_ROOT = '/kaggle/input/semvaltask8/competition/' 
qa = pd.read_csv('/kaggle/input/semvaltask8/competition/test_qa.csv') # Use 'test' split for test_qa.csv
qa_dataset = Dataset.from_pandas(qa) # Convert to Hugging Face Dataset
print(f"Loaded {len(qa_dataset)} questions.")


evaluator = Evaluator(qa=qa_dataset) # Pass the HF Dataset object

runner_lite = Runner(
    model_call=llama3_model_call,
    prompt_generator=generate_prompt_messages, # Function returning messages list
    postprocess=lambda response, dataset: postprocess_response(response, dataset),
    qa=qa_dataset, # Pass the HF Dataset object
    batch_size=4,  # Adjust based on GPU memory and desired speed vs stability
                    # Note: True batching not implemented in llama3_model_call above
)


Loaded 522 questions.


In [11]:


print("--- Running Lite Predictions ---")
responses_lite_path = "predictions_lite.txt"
responses_lite = runner_lite.run(save=responses_lite_path)
print(f"Lite predictions saved to {responses_lite_path}")



print("\n--- Evaluating ---")

if os.path.exists(responses_lite_path):
    accuracy_lite = evaluator.eval(responses_lite, lite=True)
    print(f"DataBench (Lite) Accuracy: {accuracy_lite*100:.2f}%")
else:
    print(f"ERROR: {responses_lite_path} not found. Cannot evaluate lite dataset.")

--- Running Lite Predictions ---


  0%|          | 0/131 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  1%|          | 1/131 [00:43<1:34:23, 43.56s/it]

ans:  True
ans:  True
ans:  False
ans:  False


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  2%|▏         | 2/131 [01:38<1:48:20, 50.39s/it]

ans:  True
ans:  True
ans:  True
ans:  True


...

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 99%|█████████▉| 130/131 [1:53:51<00:58, 58.76s/it]

exec error name 'List' is not defined
exec error name 'List' is not defined


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
100%|██████████| 131/131 [1:54:21<00:00, 52.38s/it]

ans:  ['Novel', 'Novel', 'Novel', 'Self-help and meditation', 'Self-help and meditation']
exec error name 'List' is not defined
Lite predictions saved to predictions_lite.txt

--- Evaluating ---


KeyError: "Column sample_answer not in the dataset. Current columns in the dataset: ['question', 'dataset']"

In [ ]:
responses_lite

**Evaluation:** 63.79% accuracy